In [ ]:
#Importing required modules
import numpy as np
import pandas as pd
import pickle
import time
import scipy
import sklearn

import feedparser as fp
import html
from nltk.tokenize import TreebankWordTokenizer,WhitespaceTokenizer,word_tokenize
from gensim.summarization import keywords
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from langdetect import detect
from website.code_bin import Cleaner
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from multiprocessing import Pool
import string
from newspaper import Article

## Models
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer


# Plotting
import matplotlib.pyplot as plt

In [ ]:
#Scan output folder for pickle files w/ identifier and clean them. 


floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/summaries4/'
foutloc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/'
identifier = '_1580326866'
file_prefix = "feeds_"
file_suffix = ".pkl"

#preprocessing wrapper function for multiprocessing
def pp(summaries):
    return (summaries[0],cleaner.preprocess_documents(summaries[1]))

cleaner = Cleaner.Cleaner()

nfeed_files = 205

if __name__ == '__main__':
    p = Pool(8)
    for i in range(nfeed_files):
        cleaned_feeds = []
        #load in file
        fname = file_prefix + str(i) + identifier + file_suffix
        with open(floc+fname,'rb') as fid:
            thisfeeds = pickle.load(fid)
        print(len(thisfeeds))
        start_time = time.time()
        cleaned_feeds = p.map(pp,thisfeeds)
        print(len(cleaned_feeds))

        with open(foutloc+'preprocessed_summaries'+str(i)+identifier+file_suffix,'wb') as fid:
            pickle.dump(cleaned_feeds,fid)

        stop_time = time.time()
        duration = stop_time - start_time
        print('Parsed file %s, duration = %.2f' % (fname,duration))
        i += 1

In [ ]:
#Combine groups of 50 files, and combine preprocessed descriptions into strings. 
#Reduces the number of files and saves space for RAM efficiency. 
import time
import pickle

floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/summaries5/'
foutloc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/summaries6/'
associator = '_1580538675'
ext = '.pkl'

file_chunk_size = 50
nfile = 126
fctr = 0
for fr in range(0,nfile,file_chunk_size):
    preprocessed_summaries = []
    for i in range(fr,min(fr+file_chunk_size,nfile-1)):

        start_time = time.time()
        

        
        fname = 'preprocessed_summaries' + str(i) + associator + ext
        with open(floc+fname,'rb') as fid:
            thisdata = pickle.load(fid)
        
        for t in thisdata:
            for j in range(0,len(t[1])):
                t[1][j] = ' '.join(t[1][j])
            preprocessed_summaries.append(t)
        
        stop_time = time.time()
        duration = stop_time - start_time
        print('loaded file %d, duration = %.2f' % (i,duration))
    print('saving chunk %d...' % fr)
    with open(foutloc + 'preprocessed_summaries' + str(fctr) + associator + ext,'wb') as fid:
        pickle.dump(preprocessed_summaries,fid)
        fctr += 1

## Loading and Preparing Data

Create a data frame where each row is a podcast and all episode descriptios for each podcasts is found in 'podcasts_eps_descriptions'. In this dataframe, stopwords and punctuations are removed, making it suitable for word2vec and doc2vec. 

In [ ]:
#Create dataset of podcasts
import pickle
import time

floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/summaries/'
fctr = 0
associator = '1580326866'
nfile = 4
episodes = []
just_podcast_vectors=[]
for i in range(0,nfile):
    start_time = time.time()
    fname = 'preprocessed_summaries' + str(fctr) +'_'+ associator + '.pkl'
    with open(floc+fname,'rb') as fid:
         pps = pickle.load(fid)
    for p in pps:
        #just_podcast_vectors.append(pod2vec(p[1]))
        episodes.append([p[0],p[1]])
    fctr += 1
    stop_time = time.time()
    duration = stop_time - start_time
    print('Done with file ' + str(i) + ' (duration=' + str(duration) + ')')

podcasts_df=pd.DataFrame(episodes)
podcasts_df.rename(columns = {0:'feedUrl',1:'podcasts_eps_descriptions'}, inplace = True)
fname = 'popular_podcasts_preprocessed' +'_'+ associator + '.pkl'
with open(floc+fname,'wb') as fid:
         pickle.dump(podcasts_df,fid)

# Preprocessing

## Preparing data for GLoVE and Word2Vec

In [ ]:
'''Create dataset of episodes'''


# First, let us load the podcasts dataframe we created in the last section.
floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/summaries6/'
associator = '1580326866'
fname = 'popular_podcasts_preprocessed' +'_'+ associator + '.pkl'
with open(floc+fname,'rb') as fid:
         podcasts_df=pickle.load(fid)
        
# Next, parse and clean the episode descriptions to prepare them for BERT (keep stopwords and punctuations).
podcasts_df['episodes_descriptions_unprocessed'] = podcasts_df.apply(lambda x:get_unprocessed_eps(x),axis=1)
podcasts_df['episodes_descriptions_processed'] = podcasts_df['episodes_descriptions_unprocessed'].apply(lambda x:get_processed_eps(x))


# Here, we create a dataframe where each row corresponds to one episode.
temp=podcasts_df.merge(podcasts_df.episodes_descriptions_processed.apply(pd.Series), left_index = True, right_index = True)
temp.drop(['episodes_descriptions_unprocesses','podcasts_eps_descriptions'], axis=1, inplace=True)
episodes_df=temp.melt(id_vars = ['feedUrl', 'episodes_descriptions_processed'], value_name = 'episodes_descriptions')
episodes_df.dropna(inplace=True)
episodes_df.reset_index(drop=True, inplace=True)

with open('episodes_glove_w2v','wb') as fid:
         pickle.dump(episodes_df,fid)

## Preparing data for Sentence BERT

In [ ]:
'''Some helper functions to process the texts'''

def get_unprocessed_eps(row):
    try:
        a=fp.parse(row.feedUrl)
        return [a['entries'][k]['content'][0]['value'] for k in range(0,len(a['entries']))]
    except:
        return None
    
    
def remove_html_tags(text):
    '''This function removes html tags from a string, e.g. <br></br>'''
    soup = BeautifulSoup(text)
    return soup.get_text()
      

def replace_newline_NBSP(text):
    '''Replaces newline characters with white space'''
    return text.replace('\n',' ').replace('\xa0',' ')


def replace_dash(text,on=True):
    '''replaces dash characters with white space. Can be turned off.'''
    if(on):
        return text.replace('-',' ')
    else:
        return text

    
def clean_BERT(text,rep_dash=True):
    return ''.join(c for c in remove_html_tags(replace_dash(replace_newline_NBSP(html.unescape(text)),rep_dash)) if 
                  c not in ['"',"'",'’','`'])

 
def get_processed_eps(eps):
    try:
        return [clean_BERT(eps[k]) for k in range(0,len(eps))]
    except:
        print('oops! couldnt clean it')
        return None



In [ ]:
#Create dataset of episodes


# First, let us load the podcasts dataframe we created in the last section.
floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/summaries6/'
associator = '1580326866'
fname = 'popular_podcasts_preprocessed' +'_'+ associator + '.pkl'
with open(floc+fname,'rb') as fid:
         podcasts_df=pickle.load(fid)
        
# Next, parse and clean the episode descriptions to prepare them for BERT (keep stopwords and punctuations).
podcasts_df['episodes_descriptions_unprocessed'] = podcasts_df.apply(lambda x:get_unprocessed_eps(x),axis=1)
podcasts_df['episodes_descriptions_processed'] = podcasts_df['episodes_descriptions_unprocessed'].apply(lambda x:get_processed_eps(x))


# Here, we create a dataframe where each row corresponds to one episode.
temp=podcasts_df.merge(podcasts_df.episodes_descriptions_processed.apply(pd.Series), left_index = True, right_index = True)
temp.drop(['episodes_descriptions_unprocesses','podcasts_eps_descriptions'], axis=1, inplace=True)
episodes_df=temp.melt(id_vars = ['feedUrl', 'episodes_descriptions_processed'], value_name = 'episodes_descriptions')
episodes_df.dropna(inplace=True)
episodes_df.reset_index(drop=True, inplace=True)

with open('processed_BERT','wb') as fid:
         pickle.dump(podcasts_df,fid)

In sentence-BERT, we find embedding for every sentence in the episode descriptions, therefore we split episode desctiptions into their constructing sentences.

In [ ]:
'''Split each episode description to sentences to prepare it to be used for Senetence-BERT'''
with open('processed_BERT','rb') as fid:
         episodes_df=pickle.load(fid)
        
episodes_df['episodes_descriptions_sentences'] = episodes_df['episodes_descriptions'].apply(lambda x: ['. '.join(x.split('. ')[0:4])])
episodes_df.drop(['episodes_descriptions_processed','episodes_descriptions'],axis=1,inplace=True)
with open('processed_BERT_first4sentences','wb') as fid:
         pickle.dump(episodes_df,fid)